In [2]:
import pandas as pd
import pycountry 

In [6]:
df_imfco = pd.read_excel(imfco,sheet_name = 'simple')

In [2]:
def read_dbs_country_group(dbsco = r'E:/workspace/lbsdot/meta/dbscountrymapping.txt'):
    dbsco = [x[:-1] for x in open(dbsco).readlines()]
    dic_ccgroup = dict()
    for i, line in enumerate(dbsco):
        if len(line.strip()) == 0:
            continue
        elif line[0] != ' ':
            is_key = True
            if i != 0:
                dic_ccgroup[grpcode] = {
                    'name': grpname,
                    'cclist': dic_cc
                }
            dic_cc = dict()
        else:
            is_key = False

        if is_key:
            grpname = line.split('[')[0].strip()
            grpcode = line.split('[')[1].strip()[:-1]
        else:
            ccname = line.split('[')[0].strip()
            cccode = line.split('[')[1].strip()[:-1]
            dic_cc[cccode] = ccname
    return dic_ccgroup

In [3]:
dic_ccgroup = read_dbs_country_group()
dic_ccgroup.keys()

dict_keys(['AEM', 'EME', 'G10', 'G20', 'OECD', 'EMU', 'EU'])

In [4]:
dic_ccgroup['G10']

{'name': 'G-10',
 'cclist': {'BE': 'Belgium',
  'CA': 'Canada',
  'FR': 'France',
  'DE': 'Germany',
  'IT': 'Italy',
  'JP': 'Japan',
  'NL': 'Netherlands',
  'SE': 'Sweden',
  'CH': 'Switzerland',
  'GB': 'United Kingdom',
  'US': 'United States'}}

In [25]:
def read_imf_country_list(imfco = r'E:/workspace/lbsdot/meta/imfcountry.xlsx'):
    df_imfco = pd.read_excel(imfco, 'simple')
    df_imfco.columns = ['imfnumeric','imfiso3','imfname']
    return df_imfco

In [26]:
df_imfco = read_imf_country_list()

In [27]:
df_imfco.head(2)

,imfnumeric,imfiso3,imfname
0,512,AFG,Afghanistan
1,914,ALB,Albania


In [20]:
df_pyco = pd.DataFrame([[x.alpha_2, x.alpha_3, x.name, x.numeric] for x in pycountry.countries], 
                      columns = ['iso2','iso3','isoname','isonumeric'])

Country(alpha_2='AW', alpha_3='ABW', name='Aruba', numeric='533')

In [23]:
len(df_imfco)

189

In [24]:
len(df_pyco)

249

In [31]:
imfco_pyco = pd.merge(df_imfco, df_pyco, left_on='imfiso3' ,right_on='iso3' )

In [32]:
imfco_pyco.head(2)

,imfnumeric,imfiso3,imfname,iso2,iso3,isoname,isonumeric
0,512,AFG,Afghanistan,AF,AFG,Afghanistan,004
1,914,ALB,Albania,AL,ALB,Albania,008


In [41]:
imfco_pyco[imfco_pyco.iso2.isin([x for x in dic_ccgroup['G10']['cclist']])]

,imfnumeric,imfiso3,imfname,iso2,iso3,isoname,isonumeric
15,124,BEL,Belgium,BE,BEL,Belgium,056
30,156,CAN,Canada,CA,CAN,Canada,124
57,132,FRA,France,FR,FRA,France,250
61,134,DEU,Germany,DE,DEU,Germany,276
80,136,ITA,Italy,IT,ITA,Italy,380
82,158,JPN,Japan,JP,JPN,Japan,392
118,138,NLD,Netherlands,NL,NLD,Netherlands,528
159,144,SWE,Sweden,SE,SWE,Sweden,752
160,146,CHE,Switzerland,CH,CHE,Switzerland,756
177,112,GBR,United Kingdom,GB,GBR,United Kingdom,826


### loading into Mongodb 

In [44]:
from pymongo import MongoClient

In [62]:
mongoclient = MongoClient(unicode_decode_error_handler='ignore')

In [49]:
mongoclient.lbsdot.countrytable.insert_many(imfco_pyco.to_dict('records'))

In [63]:
# Coutry group on DBSonline
for x in dic_ccgroup:
    raw = {'code': x}
    raw.update(dic_ccgroup[x])
    mongoclient.lbsdot.countrygroup.insert_one(raw)

In [64]:
mongoclient.close()

### Adding longitude and latitude

In [3]:
cc_lat_lng = pd.read_csv(r'E:/workspace/lbsdot/data/country_lat_lng.txt',sep='\t')

In [8]:
cc_lat_lng.head()

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [4]:
from pymongo import MongoClient
mongoclient = MongoClient(unicode_decode_error_handler='ignore')

In [9]:
for x in cc_lat_lng.apply(lambda x: (x.country, x.latitude, x.longitude), axis=1):
    _query = {"iso2": x[0]}
    _newcols = {"$set":{"lat": x[1], "lng": x[2]}}
    mongoclient.lbsdot.countrytable.update_one(_query, _newcols)

In [12]:
pd.DataFrame(list(mongoclient.lbsdot.countrytable.find())).head()

,_id,imfnumeric,imfiso3,imfname,iso2,iso3,isoname,isonumeric,lat,lng
0,5d3aac08126a14cce2420890,512,AFG,Afghanistan,AF,AFG,Afghanistan,004,33.939110,67.709953
1,5d3aac08126a14cce2420891,914,ALB,Albania,AL,ALB,Albania,008,41.153332,20.168331
2,5d3aac08126a14cce2420892,612,DZA,Algeria,DZ,DZA,Algeria,012,28.033886,1.659626
3,5d3aac08126a14cce2420893,614,AGO,Angola,AO,AGO,Angola,024,-11.202692,17.873887
4,5d3aac08126a14cce2420894,311,ATG,Antigua and Barbuda,AG,ATG,Antigua and Barbuda,028,17.060816,-61.796428
